In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [3]:
config_file = "configs/train_tdt4265.yaml"

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2021-04-25 11:34:23,112 SSD INFO: Loaded configuration file configs/train_tdt4265.yaml
2021-04-25 11:34:23,113 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    BACKBONE:
        NAME: 'resnet18'
        PRETRAINED: True
        #OUT_CHANNELS: [512, 1024, 2048, 1024, 1024, 512] # resnet152
        OUT_CHANNELS: [128, 256, 512, 1024, 512, 512] # resnet18
        #OUT_CHANNELS: [512, 1024, 512, 512, 256, 256] # model from assignment 4
        INPUT_CHANNELS: 3
    PRIORS:
        # Image size [300, 300]
        #FEATURE_MAPS: [[38, 38], [19, 19], [10, 10], [5, 5], [3, 3], [1, 1]]
        #STRIDES: [[8, 8], [16, 16], [32, 32], [64, 64], [100, 100], [300, 300]]
        #MIN_SIZES: [[30, 30], [60, 60], [111, 111], [162, 162], [213, 213], [264, 264]]
        #MAX_SIZES: [[60, 60], [111, 111], [162, 162], [213, 213], [264, 264], [315, 315]]

        # Image size [600, 600]
        #FEATURE_MAPS: [[75, 75], [38, 38], [19, 19], [10, 10], [5, 5], [3, 3]]
        #MIN_SIZES: [[60, 60], [120, 120], [222, 2

In [ ]:
model = start_train(cfg)

Detector initialized. Total Number of params:  49.01M
Backbone number of parameters: 47.76M
SSD Head number of parameters: 1.24M
2021-04-25 11:35:47,730 SSD.trainer INFO: Loading checkpoint from outputs/rdd2020/model_072500.pth
2021-04-25 11:35:48,033 SSD.trainer INFO: Loading optimizer from outputs/rdd2020/model_072500.pth
Dataset loaded. Subset: train, number of images: 19094
2021-04-25 11:35:52,624 SSD.trainer INFO: Start training ...


/home/fredralm/home/TDT4265-StarterCode/Project/SSD/ssd/data/transforms/transforms.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/fredralm/home/TDT4265-StarterCode/Project/SSD/ssd/data/transforms/transforms.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/fredralm/home/TDT4265-StarterCode/Project/SSD/ssd/data/transforms/transforms.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of list

2021-04-25 11:35:58,278 SSD.trainer INFO: iter: 072510, lr: 0.00050, total_loss: 3.792 (3.792), reg_loss: 1.301 (1.301), cls_loss: 2.491 (2.491), time: 0.565 (0.565), eta: 7:26:53, mem: 6169M
2021-04-25 11:36:02,075 SSD.trainer INFO: iter: 072520, lr: 0.00050, total_loss: 3.477 (3.634), reg_loss: 1.116 (1.209), cls_loss: 2.361 (2.426), time: 0.380 (0.472), eta: 6:13:40, mem: 6169M
2021-04-25 11:36:06,309 SSD.trainer INFO: iter: 072530, lr: 0.00050, total_loss: 3.346 (3.538), reg_loss: 1.079 (1.165), cls_loss: 2.267 (2.373), time: 0.423 (0.456), eta: 6:00:42, mem: 6169M
2021-04-25 11:36:09,716 SSD.trainer INFO: iter: 072540, lr: 0.00050, total_loss: 3.477 (3.523), reg_loss: 1.100 (1.149), cls_loss: 2.376 (2.374), time: 0.341 (0.427), eta: 5:37:50, mem: 6169M
2021-04-25 11:36:13,817 SSD.trainer INFO: iter: 072550, lr: 0.00050, total_loss: 3.471 (3.513), reg_loss: 1.119 (1.143), cls_loss: 2.352 (2.369), time: 0.410 (0.424), eta: 5:35:05, mem: 6169M
2021-04-25 11:36:17,373 SSD.trainer INFO

In [5]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)

2021-04-21 18:09:34,587 SSD INFO: Start evaluating...


NameError: name 'model' is not defined